In [1]:
from torchvision.models.resnet import ResNet, BasicBlock

In [2]:
import pandas as pd
from JigsawDataLoader import *
from JigsawNetwork import Network, AlexNet
from fastprogress.fastprogress import master_bar, progress_bar

import torch
import torch.nn as nn
import torch.nn.functional as F

gpu_id = 4
num_cores = 4
torch.cuda.set_device(gpu_id)

In [3]:
class ResNet34(ResNet):
    
    def __init__(self, num_classes=24):
        super().__init__(block=BasicBlock, layers=[3, 4, 6, 3], num_classes=num_classes)
        self.fc = nn.Linear(512 * BasicBlock.expansion * 4, num_classes)
        
    def _forward_impl(self, z):
        # See note [TorchScript super()]
        B,T,C,H,W = z.size()
        z = z.transpose(0,1)

        z_list = []
        for i in range(4):        
            x = z[i]
            x = self.conv1(x)
            x = self.bn1(x)
            x = self.relu(x)
            x = self.maxpool(x)

            x = self.layer1(x)
            x = self.layer2(x)
            x = self.layer3(x)
            x = self.layer4(x)
            x = self.avgpool(x)
            
            z_list.append(x)

        x = torch.cat(z_list, 1)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x):
        return self._forward_impl(x)

In [4]:
def subsample_dl(ds, bs=16, pct=0.1, seed=None):
    """Takes a DataLoader object as input and returns a mini-version of the dataset
    This is useful for debugging and rapid experimentation. 
    data -> a databunch object
    pct  -> the fraction of original dataset size (default: 0.1)"""
    import numpy as np
    if seed: np.random.seed(seed)
    size = len(ds)
    indices = np.random.choice(np.arange(size), 
                                   size=int(pct*size), replace=False)
    sampler = torch.utils.data.sampler.SubsetRandomSampler(indices)
    mini_dl = torch.utils.data.DataLoader(dataset=ds,
                                            batch_size=bs,
                                            sampler=sampler,
                                            num_workers=4)
    return mini_dl

#### train args

In [12]:
bs = 16
img_size = 256
patch_size = 180
subsample = True

In [13]:
train_pth = pd.read_csv('../train.csv')
train_ds = DataLoader(train_pth, img_size=img_size, patch_size=patch_size)
train_loader = torch.utils.data.DataLoader(dataset=train_ds,
                                            batch_size=bs,
                                            shuffle=True,
                                            num_workers=num_cores)
sub_train_loader = subsample_dl(train_ds, bs)

In [14]:
val_pth = pd.read_csv('../valid.csv')
val_ds = DataLoader(val_pth, img_size=img_size, patch_size=patch_size)
val_loader = torch.utils.data.DataLoader(dataset=val_ds,
                                            batch_size=bs,
                                            shuffle=True,
                                            num_workers=num_cores)
sub_val_loader = subsample_dl(val_ds, bs)

In [15]:
if subsample:
    train_data = sub_train_loader
    val_data = sub_val_loader
else:
    train_data = train_loader
    val_data = val_loader

In [22]:
net = ResNet34()
# net = AlexNet()
net = net.cuda()

In [23]:
# loss func and optimizer
lr = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(),lr=lr,momentum=0.9,weight_decay = 1e-3)

In [64]:
x, y = train_ds[0]
x = x.unsqueeze(0).cuda()

In [69]:
y = torch.LongTensor([y]).cuda()

In [66]:
y

tensor([13.], device='cuda:4')

In [67]:
output = net(x)

In [70]:
criterion(input=output, target=y)

tensor(3.6853, device='cuda:4', grad_fn=<NllLossBackward>)

In [72]:
output

tensor([[-0.0325,  0.2702,  0.7118,  0.5624,  1.0814,  0.3620, -0.0683, -0.4219,
         -0.2902,  0.0297, -1.5623, -0.3395, -0.4992, -0.3150,  0.0336,  0.6344,
         -0.6200,  0.1076,  0.8112, -0.6467,  0.7306, -1.0499,  0.4051,  0.5496]],
       device='cuda:4', grad_fn=<AddmmBackward>)

In [73]:
y

tensor([13], device='cuda:4')

In [74]:
batch_time, net_time = [], []
for epoch in range(10):
#         if epoch%10==0 and epoch>0:
#             test(net,criterion,logger_test,val_loader,steps)
#         lr = adjust_learning_rate(optimizer, epoch, init_lr=args.lr, step=20, decay=0.1)
        net.train()
        total_loss = 0
        total_items = 0
        for i, (images, labels) in enumerate(progress_bar(train_data)):
            images = images.cuda()
            labels = labels.cuda()
            
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(input=outputs, target=labels)
            print(loss)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()*len(labels)
            total_items += len(labels)
        train_loss = total_loss / total_items
        print(f'Train loss {train_loss:.2f}')

tensor(3.4317, device='cuda:4', grad_fn=<NllLossBackward>)
tensor(3.3101, device='cuda:4', grad_fn=<NllLossBackward>)
tensor(3.1637, device='cuda:4', grad_fn=<NllLossBackward>)
tensor(2.9611, device='cuda:4', grad_fn=<NllLossBackward>)
tensor(3.1469, device='cuda:4', grad_fn=<NllLossBackward>)
tensor(3.0527, device='cuda:4', grad_fn=<NllLossBackward>)
tensor(2.9327, device='cuda:4', grad_fn=<NllLossBackward>)
tensor(2.7217, device='cuda:4', grad_fn=<NllLossBackward>)
tensor(nan, device='cuda:4', grad_fn=<NllLossBackward>)
tensor(nan, device='cuda:4', grad_fn=<NllLossBackward>)
tensor(nan, device='cuda:4', grad_fn=<NllLossBackward>)
tensor(nan, device='cuda:4', grad_fn=<NllLossBackward>)
tensor(nan, device='cuda:4', grad_fn=<NllLossBackward>)
tensor(nan, device='cuda:4', grad_fn=<NllLossBackward>)
tensor(nan, device='cuda:4', grad_fn=<NllLossBackward>)
tensor(nan, device='cuda:4', grad_fn=<NllLossBackward>)
tensor(nan, device='cuda:4', grad_fn=<NllLossBackward>)
tensor(nan, device='cuda

KeyboardInterrupt: 

In [18]:
def valid_metrics(model, valid_dl):
    model.eval()
    total_loss = 0
    total_true = 0
    total_items = 0
    for images, y in valid_dl:
        images = images
        images = images.cuda()
        out = model(images)
        out = out.cpu().data
        loss = F.cross_entropy(out, y)
        total_loss += loss.item()*len(y)
        total_true += (out.argmax(dim=1) == y).sum().item()
        total_items += len(y)
    val_loss = total_loss / total_items
    val_acc = total_true / total_items
    print(f'Valid loss {val_loss:.2f}')
    print(f'Accuracy: {val_acc:.2f}')
    return val_loss, val_acc

In [19]:
valid_metrics(net, val_data)

Valid loss 3.19
Accuracy: 0.06


(3.192456855773926, 0.056)

# Original script

In [ ]:
import os, sys, numpy as np
import argparse
from time import time
from tqdm import tqdm
from torch.autograd import Variable

# import tensorflow # needs to call tensorflow before torch, otherwise crush #???
# sys.path.append('Utils')
# from logger import Logger
# from TrainingUtils import adjust_learning_rate, compute_accuracy

parser = argparse.ArgumentParser(description='Train JigsawPuzzleSolver on Imagenet')
parser.add_argument('data', type=str, help='Path to Imagenet folder')
parser.add_argument('--model', default=None, type=str, help='Path to pretrained model')
parser.add_argument('--classes', default=1000, type=int, help='Number of permutation to use')
parser.add_argument('--gpu', default=0, type=int, help='gpu id')
parser.add_argument('--epochs', default=70, type=int, help='number of total epochs for training')
parser.add_argument('--iter_start', default=0, type=int, help='Starting iteration count')
parser.add_argument('--batch', default=256, type=int, help='batch size')
parser.add_argument('--checkpoint', default='checkpoints/', type=str, help='checkpoint folder')
parser.add_argument('--lr', default=0.001, type=float, help='learning rate for SGD optimizer')
parser.add_argument('--cores', default=0, type=int, help='number of CPU core for loading')
parser.add_argument('-e', '--evaluate', dest='evaluate', action='store_true',
                    help='evaluate model on validation set, No training')
args = parser.parse_args()

In [ ]:
def main():
# data loader
    train_data = DataLoader(trainpath,args.data+'/ilsvrc12_train.txt',
                            classes=args.classes)
    train_loader = torch.utils.data.DataLoader(dataset=train_data,
                                            batch_size=args.batch,
                                            shuffle=True,
                                            num_workers=args.cores)
    
    valpath = args.data+'/ILSVRC2012_img_val'
    if os.path.exists(valpath+'_255x255'):
        valpath += '_255x255'
    val_data = DataLoader(valpath, args.data+'/ilsvrc12_val.txt',
                            classes=args.classes)
    val_loader = torch.utils.data.DataLoader(dataset=val_data,
                                            batch_size=args.batch,
                                            shuffle=True,
                                            num_workers=args.cores)
# iterating through data
    
    N = train_data.N
    
    iter_per_epoch = train_data.N/args.batch
    print('Images: train %d, validation %d'%(train_data.N,val_data.N))
    
# Network initialize
    net = Network(args.classes)
    if args.gpu is not None:
        net.cuda()
    
    ############## Load from checkpoint if exists, otherwise from model ###############
    if os.path.exists(args.checkpoint):
        files = [f for f in os.listdir(args.checkpoint) if 'pth' in f]
        if len(files)>0:
            files.sort()
            #print files
            ckp = files[-1]
            net.load_state_dict(torch.load(args.checkpoint+'/'+ckp))
            args.iter_start = int(ckp.split(".")[-3].split("_")[-1])
            print('Starting from: ',ckp)
        else:
            if args.model is not None:
                net.load(args.model)
    else:
        if args.model is not None:
            net.load(args.model)

# loss func and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(),lr=args.lr,momentum=0.9,weight_decay = 5e-4)
    
# dunno how to use the logger yet
#     logger = Logger(args.checkpoint+'/train')
#     logger_test = Logger(args.checkpoint+'/test')
    
    ############## TESTING ###############
    if args.evaluate:
        test(net,criterion,None,val_loader,0)
        return
    
    ############## TRAINING ###############
    print(('Start training: lr %f, batch size %d, classes %d'%(args.lr,args.batch,args.classes)))
    print(('Checkpoint: '+args.checkpoint))
    
    # Train the Model
    batch_time, net_time = [], []
    steps = args.iter_start
    for epoch in range(int(args.iter_start/iter_per_epoch),args.epochs):
        if epoch%10==0 and epoch>0:
            test(net,criterion,logger_test,val_loader,steps)
        lr = adjust_learning_rate(optimizer, epoch, init_lr=args.lr, step=20, decay=0.1)
        
        end = time()
        for i, (images, labels, original) in enumerate(train_loader):
            batch_time.append(time()-end)
            if len(batch_time)>100:
                del batch_time[0]
            
            images = Variable(images)
            labels = Variable(labels)
            if args.gpu is not None:
                images = images.cuda()
                labels = labels.cuda()

            # Forward + Backward + Optimize
            optimizer.zero_grad()
            t = time()
            outputs = net(images)
            net_time.append(time()-t)
            if len(net_time)>100:
                del net_time[0]
            
            prec1, prec5 = compute_accuracy(outputs.cpu().data, labels.cpu().data, topk=(1, 5))
            acc = prec1[0]

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            loss = float(loss.cpu().data.numpy())

            if steps%20==0:
                print(('[%2d/%2d] %5d) [batch load % 2.3fsec, net %1.2fsec], LR %.5f, Loss: % 1.3f, Accuracy % 2.2f%%' %(
                            epoch+1, args.epochs, steps, 
                            np.mean(batch_time), np.mean(net_time),
                            lr, loss,acc)))

            if steps%20==0:
                logger.scalar_summary('accuracy', acc, steps)
                logger.scalar_summary('loss', loss, steps)
                
                original = [im[0] for im in original]
                imgs = np.zeros([9,75,75,3])
                for ti, img in enumerate(original):
                    img = img.numpy()
                    imgs[ti] = np.stack([(im-im.min())/(im.max()-im.min()) 
                                         for im in img],axis=2)
                
                logger.image_summary('input', imgs, steps)

            steps += 1

            if steps%1000==0:
                filename = '%s/jps_%03i_%06d.pth.tar'%(args.checkpoint,epoch,steps)
                net.save(filename)
                print('Saved: '+args.checkpoint)
            
            end = time()

        if os.path.exists(args.checkpoint+'/stop.txt'):
            # break without using CTRL+C
            break

In [ ]:
def test(net,criterion,logger,val_loader,steps):
    print('Evaluating network.......')
    accuracy = []
    net.eval()
    for i, (images, labels, _) in enumerate(val_loader):
        images = Variable(images)
        if args.gpu is not None:
            images = images.cuda()

        # Forward + Backward + Optimize
        outputs = net(images)
        outputs = outputs.cpu().data

        prec1, prec5 = compute_accuracy(outputs, labels, topk=(1, 5))
        accuracy.append(prec1[0])

    if logger is not None:
        logger.scalar_summary('accuracy', np.mean(accuracy), steps)
    print('TESTING: %d), Accuracy %.2f%%' %(steps,np.mean(accuracy)))
    net.train()